# Set Up for Project Imports


In [ ]:
# import sys
# from pathlib import Path

In [ ]:
# source_directory = Path.cwd()
# ROOT = source_directory.parent
# if str(ROOT) not in sys.path:
#     sys.path.insert(0, str(ROOT))

In [ ]:
# # Auto-reload code changes
# %load_ext autoreload
# %autoreload 2

# Imports

In [ ]:
from data.api import UcIrvineAPI, UcIrvineDatasetIDs, BureauEconomicAnalysisAPI
import data.wrangling_utils
import pandas

In [ ]:
pandas.set_option('display.max_colwidth', None)  # show all text in cells
# pandas.set_option("display.max_rows", 100_000)
pandas.options.mode.copy_on_write = True
pandas.set_option('display.float_format', lambda x: '%.2f' % x)

# UcIrvine Data

In [ ]:
uci = UcIrvineAPI.fetch_dataset(repo_id=UcIrvineDatasetIDs.Apartment_For_Rent_Classified.value)

In [ ]:
uci_df: pandas.DataFrame = uci.data.original.reset_index()

In [ ]:
clean_uci_df: pandas.DataFrame = data.wrangling_utils.clean(uci_df)

In [ ]:
# DO NOT DELETE MIGHT NEED
# pandas.set_option("display.max_rows", 100_000)  # TOGGLE  UN/COMMENT
# pandas.reset_option("display.max_rows") # TOGGLE UN/COMMENT
# cleaned_uci_df['cityname'].value_counts(dropna=False)  # change column

In [ ]:
# DO NOT DELETE MIGHT NEED
# import json
# s = cleaned_uci_df['bathrooms'].explode()
# global_counts = s.value_counts().to_dict()
# global_counts
# print(f'BAD_DATA: {json.dumps(BAD_DATA['cityname'], indent=2)}')  # CHANGE COL
# uci_df["state_full"] = uci_df["state"].str.upper().map(STATE_MAP)
# print(uci_df.shape)
# uci_df.dropna(subset=["state_full"], inplace=True)
# uci_df.shape

# Bureau of Economic Data

In [ ]:
bea_df = BureauEconomicAnalysisAPI.fetch_dataset('Regional', GeoFips='STATE', TableName='SARPP', Year='2019',
                                                 LineCode='1')